# Introducción a la recuperación de información

La recuperación de información se crea a partir de la necesidad de simplificar el acceso y revisión de documentos en grandes colecciones. Estas colecciones pueden ser homogeneas o heterogeneas, tanto en su contenido como en su formato. En su inicio, se consideraban colecciones unicamente de texto pero las necesidades de información se han diversificado, y ahora es común encontrar sistemas de recuperación de información sobre otros datos como imagenes o videos, o include multimodales, esto es que puedan usar diferentes tipos de objetos. Un objeto puede ser un documento de texto, una imagen, o cualquier otro tipo de datos que se desee tener acceso. 

Mantener un sistema de información homogeneo puede simplificar su mantenimiento enormemente, por lo que si es posible, se puede intentar mantener cierta homegeneidad. Para sistemas de fuentes abiertas como puede ser la web, esto será posible.

En general, se puede ver un sistema de recuperación de información en tres grandes partes:

1. Recolección de datos, normalización y modelado matemático.
2. Indexamiento, interpretación de consultas y optimización
3. Agregación y filtrado de resultados, presentación de los mismos


En este curso se visitarán parcialmente todas estas partes. En general se usaran conjuntos de datos previamente recolecados, aunque se invita a usar sus propias colecciones de tal forma que puedan sacar provecho de los temas de manera más amplia. La normalización puede ir desde el simple preprocesamiento de los datos hasta manipulaciones y transformaciones dependientes del dominio. Una vez aplicado el modelado adecuado a las colecciones, las representaciones matemáticas suelen ser vectores de alta dimensión para cada objeto.

En cuanto al indexamiento se utilizarán dos tipos de algoritmos, búsqueda mediante índices invertidos y búsqueda por índices métricos. Ambas tienen sus nichos de aplicación y usarlas adecuadamente requiere conocer los problemas y sus modelados.

La presentación de los resultados puede ser tan simple como una lista de resultados más relevantes y una pequeña muestra del objeto, o más complejo que requiera alguna técnica de visualización. Todo esto dependerá del dominio de aplicación y la naturaleza del sistema de recuperación de información.


# Ejemplos de sistemas de recuperación información

- Google,scholar google, google images, google news, youtube...
- Bing!
- Yahoo search
- Yandex
- DuckDuckGo
- ...

También es común que el sistema de recuperación de información no sea el principal producto, si no más bien un complemento imperdible
como es el caso de los sistemas de streaming de videos, películas, música, redes sociales, etc.

# Ejemplos de implementación de search engines open source
- [Apache Lucene](https://lucene.apache.org/)
- [Apache SOLR](https://solr.apache.org/)
- [ElasticSearch](https://www.elastic.co/es/what-is/elasticsearch)
- [hnswlib](https://github.com/nmslib/hnswlib)
- [FAISS](https://github.com/facebookresearch/faiss)
- [TextSearch.jl](https://github.com/sadit/TextSearch.jl)
- [InvertedFiles.jl](https://github.com/sadit/InvertedFiles.jl)
- [SimilaritySearch.jl](https://github.com/sadit/SimilaritySearch.jl)


# Ejemplo


Construcción de un índice de búsqueda para una colección de mensajes de Twitter. Los mensajes se pueden ver como documentos cortos. Se preprocesan y se modalen en vectores mediante un vocabulario. Después se construye el índice con dichos vectores y se hace posible la búsqueda eficiente. Note que existen etiquetas que se pueden usar para otras tareas; de manera más genérica, se pueden ver como metadatos asociados a los elementos del corpus.

In [1]:
using Pkg
Pkg.activate(".")

!isfile("Manifest.toml") && Pkg.add([
    PackageSpec(name="TextSearch", version="0.12"),
    PackageSpec(name="InvertedFiles", version="0.4"),
    PackageSpec(name="CodecZlib", version="0.7"),
    PackageSpec(name="JSON", version="0.21"),
    PackageSpec(name="HypertextLiteral", version="0.9")
])

using TextSearch, InvertedFiles, SimilaritySearch, TextSearch, CodecZlib, JSON, LinearAlgebra, HypertextLiteral

using Downloads: download

  Activating project at `~/courses/IR-2022/U1`


## Funciones para leer y modelar el corpus

In [2]:
function parse_corpus(corpusfile)
    corpus, labels = String[], String[]
    open(corpusfile) do f
        for line in eachline(GzipDecompressorStream(f))
            r = JSON.parse(line)
            push!(labels, r["klass"])
            push!(corpus, r["text"])    
        end
    end
    
    corpus, labels
end

function text_model_and_vectors(corpus)
    textconfig = TextConfig(group_usr=true, group_url=true, del_diac=true, lc=true, group_num=true, nlist=[1], qlist=[])
    model = VectorModel(IdfWeighting(), TfWeighting(), textconfig, corpus)    
    vectors = vectorize_corpus(model, textconfig, corpus)
    for v in vectors
        normalize!(v)
    end

    (; textconfig, model, vectors)
end

function create_dataset(corpusfile)
    corpus, labels = parse_corpus(corpusfile)
    (; labels, corpus, text_model_and_vectors(corpus)...)
end

create_dataset (generic function with 1 method)

In [19]:
display(@htl "<h1>Cargando el corpus</h1>")
dbfile = "emo50k.json.gz"
baseurl = "https://github.com/sadit/TextClassificationTutorial/blob/main/data"

!isfile(dbfile) && download("$baseurl/$dbfile?raw=true", dbfile)
D = create_dataset(dbfile)

@show unique(D.labels), D.model

<h1>Cargando el corpus</h1>

(unique(D.labels), D.model) = (["😰", "😥", "😊", "😏", "♡", "💔", "🙂", "😋", "😌", "🌚", "👌", "😪", "😤", "🙃", "🤤", "😴", "😢", "😅", "😑", "😠", "😂", "😜", "🤓", "💙", "😀", "🤗", "🤣", "😒", "✨", "😐", "😞", "😁", "😱", "👏", "😫", "😍", "❤", "😣", "🙊", "🙏", "🙄", "🤭", "💜", "🤔", "😬", "👀", "😉", "😈", "😡", "😳", "🙈", "😻", "😔", "😓", "💕", "🎶", "😭", "😕", "♥", "💖", "😎", "😘", "😃", "😩"], {VectorModel global_weighting=IdfWeighting(), local_weighting=TfWeighting(), train-voc=45374, train-n=50000, maxoccs=93991})


(["😰", "😥", "😊", "😏", "♡", "💔", "🙂", "😋", "😌", "🌚"  …  "💕", "🎶", "😭", "😕", "♥", "💖", "😎", "😘", "😃", "😩"], {VectorModel global_weighting=IdfWeighting(), local_weighting=TfWeighting(), train-voc=45374, train-n=50000, maxoccs=93991})

In [4]:
invfile = WeightedInvertedFile(length(D.model.voc))
append!(invfile, VectorDatabase(D.vectors))

{WeightedInvertedFile vocsize=45374, n=50000}

In [15]:
function search_and_display(invfile, D, q, k)
    res = KnnResult(k)
    @time search(invfile, vectorize(D.model, D.textconfig, q), res)
    L = []
    for (i, (id, dist)) in enumerate(res)
        dist = round(dist, digits=2)
        push!(L, @htl """<tr><td>$i</td> <td>$id => $dist</td> <td>$(D.labels[id])</td> <td>$(D.corpus[id])</td> </tr>""")
       # println(i, " ", id => round(dist, digits=2), " ", D.labels[i], " ", D.corpus[id])
    end

    display(@htl """
    <h3>Resultados para la consulta "$q"</h3>
    <table>
    $L
    </table>
    """)
end

search_and_display (generic function with 2 methods)

In [18]:
search_and_display(invfile, D, "no importa distancia", 5)
search_and_display(invfile, D, "cuando nos vemos?", 5)
search_and_display(invfile, D, "feliz cumpleaños?", 3)
search_and_display(invfile, D, "ola k ase", 3)
search_and_display(invfile, D, "mi humilde opinión", 10)

  0.000294 seconds (21 allocations: 1.594 KiB)


1,39550 => 0.03,💕,No importa la distancia _emo _usr _url
2,3870 => 0.38,😓,Me caga la distancia _emo
3,11115 => 0.52,😃,No importa cuando... _emo _emo _url
4,38044 => 0.57,😥,"tiempo y distancia, quizás sea eso... _emo"
5,12728 => 0.57,😑,_usr A ti no te importa _emo


  0.000276 seconds (22 allocations: 1.609 KiB)


1,41790 => 0.22,😂,Nos vemos!! _emo _emo _emo _emo _url
2,27642 => 0.3,😜,_usr En la tarde nos vemos _emo _emo _emo
3,5656 => 0.32,😻,Nos vemos pronto _usr _usr _emo _emo _url
4,2835 => 0.43,😘,_usr Muchas gracias! Nos vemos pronto _emo
5,44970 => 0.43,😍,_usr Pronto nos vemos hermana _emo _emo


  0.000160 seconds (21 allocations: 1.578 KiB)


1,37575 => 0.09,❤,_usr Feliz Cumpleaños _emo
2,41849 => 0.09,😁,Feliz Cumpleaños _emo _emo _usr
3,12769 => 0.09,🙊,Feliz Cumpleaños _usr _emo _emo _emo _emo _emo


  0.000017 seconds (20 allocations: 1.422 KiB)


1,23340 => 0.64,😎,"_usr _usr Siempre, todo y todos los días de la semana desde ase muchoooooossss años _emo _emo"
2,6952 => 0.7,😤,_usr k bueno k tu mizmo lo reconozcAszzzz _emo
3,1874 => 0.72,😃,"Despertares!! (By OLA) # _emo _emo en Distrito Federal, Mexico _url"


  0.000238 seconds (20 allocations: 1.516 KiB)


1,19966 => 0.32,🙏,"Bueno, cada quien su vida, ya se, solo es una humilde opinión !! _emo _emo _emo"
2,7756 => 0.5,😍,_usr es la persona más humilde que conozco _emo
3,4755 => 0.62,🤔,_usr Bajo tu opinión fue o no falta? _emo
4,24948 => 0.62,😊,Gracias por compartir la información y tu opinión _emo _url
5,20021 => 0.64,😉,Cuantos Likes para mi humilde oficina _usr _emo _emo @ CocoBongo _url
6,34164 => 0.66,🤗,"Para ser feliz, no escuches la opinión de los demás _emo"
7,47417 => 0.68,👏,_usr Totalmente de acuerdo contigo en todo. Comparto tu opinión _emo _emo _emo _emo
8,846 => 0.69,😌,Cambie de opinión quiero seguir siendo rubia _emo _emo _emo _emo _emo _emo
9,15711 => 0.72,😌,"Tu opinión no me interesa más. No tengo nada que demostrarte, y si piensas lo peor de mi, MEJOR! _emo"
10,9033 => 0.72,😑,"Lo siento pero mi tez humilde no me deja ser guapo y estar bueno, eso sin mencionar la rodilla que me chingue! _emo _emo"


## También es posible consultar elementos del mismo corpus, lo cual permité inspeccionar la estructura interna de la colección

In [20]:
for i in 1:10
    qID = rand(1:length(D.corpus))
    search_and_display(invfile, D, D.corpus[qID], 7)
end

  0.005593 seconds (79 allocations: 7.812 KiB)


1,29434 => 0.53,🙏,"_emo _emo _emoAlfombra Roja, del Aniversario de América Salsa Mexico. Y Show en vivo del mismo Certamen con Orquesta a Bordo. Gloria a #DIOS _emo _emo _emo _emo _emo _emo _emo #GOD #CRISTOVIVE #Salsa #Mexico #Festival… _url"
2,28560 => 0.75,😭,No mi papá no. _emo _emo _emo
3,39102 => 0.75,♥,Te amo papá. _emo _emo _emo _emo _emo _emo _emo _emo _emo _emo
4,6090 => 0.79,😰,Ya quiero ver a mi papá _emo
5,2002 => 0.81,🌚,Acabo de despertar y es la gloria _emo
6,41498 => 0.82,😞,No se que pensar Papá _emo. _url
7,3786 => 0.82,😬,El_usr perfecto para mi papá!! _emo


  0.001623 seconds (49 allocations: 4.578 KiB)


1,12892 => 0.71,😃,_emo _emo aaaa jajajaja!! _url
2,7363 => 0.77,🤣,Aaaa no te pases de verga _emo
3,43202 => 0.78,👀,_usr Aaaa meño. A ver _emo
4,18209 => 0.8,✨,Siempre voy amar los atardeceres _emo _emo . . #Verbena_num en Verbena Culiacan _url
5,44961 => 0.8,😢,Aaaa mugres semaforos _emo
6,19073 => 0.81,😴,"De las casas de las esquinas, a las casas de las montañas. Buenas noches _emo _url"
7,20259 => 0.82,❤,"Al Fin!!! Para Todo Mal Mezcal Y Para Todo Bien, También _emo (@ La Verbena in Playa del Carmen, Quintana Roo) _url"


  0.004693 seconds (76 allocations: 6.531 KiB)


...

1,43030 => 0.7,😈,"Cuando mi papá dice no, mi mamá dice sí _emo _emo"
2,35232 => 0.75,👀,A ver que dice _emo
3,13112 => 0.75,😐,Dice mi mamá que siempre no _emo
4,10171 => 0.76,🙄,_usr Jajajaja así dice la canción _emo
5,27895 => 0.78,😒,Siempre dice eso _emo _url
6,48894 => 0.78,😀,El amor de mi vida me dice mamá _emo
7,35937 => 0.78,😴,Se cancela mi salida de hoy. _emo


  0.001967 seconds (74 allocations: 6.531 KiB)


1,29553 => 0.72,💙,Jueveeeeeess _emo
2,42870 => 0.78,🤗,Habemus vestido _emo _emo _emo
3,328 => 0.79,😣,Quiero abrazarte ya _emo
4,36485 => 0.79,💜,'Toni Erdmann' superó mis expectativas. Maren Ade juega magistralmente con la comedia y la crisis. La secuencia de la fiesta lo vale todo _emo
5,15561 => 0.79,🤤,HABEMUS LIGUILLA _emo _emo _emo _emo
6,20453 => 0.79,🤭,"Pues minimo, pq agraciada NO eres jajajaja _emo"
7,1535 => 0.82,😡,Qué calor _emo _emo _emo


  0.000170 seconds (32 allocations: 2.250 KiB)


1,11302 => 0.64,😥,#GYMtime _emo _emo #MeEstoyMuriendo #ElQuePerseveraAlcanza ¡De verdad estoy… _url
2,30526 => 0.74,😓,Se les acabo su gordito jajaja #pierna #gluteos #gymtime #empezandode_num _emo _emo @ Gym _url
3,41771 => 0.74,😎,Mañana empieza la verdadera chinga de inicio de año!!!! #gymtime #fitnessmotivation #dieta #proteina... Voy con todo _emo _emo _emo
4,6030 => 0.77,😣,Un tweet después _emo _emo _url
5,47611 => 0.78,😋,"después de todo lo que hice hoy, me me merecía una cena deliciosa como la que me hice! _emo _emo"
6,24286 => 0.79,👌,"One more experience... _emo _emo _emo _emo _emo _emo _emo _emo _emo @ Acapulco, Guerrero _url"
7,23612 => 0.79,😂,Siempre despues de la peda _emo _emo _url
